[View in Colaboratory](https://colab.research.google.com/github/HUSTneeb/GoogleColabCode/blob/master/TensorFlowLearning11.ipynb)

In [1]:
import tensorflow as tf
import tensorflow.feature_column as fc

import os
import sys

import matplotlib.pyplot as plt
from IPython.display import clear_output

tf.enable_eager_execution

<function tensorflow.python.framework.ops.enable_eager_execution>

In [2]:
!pip install -q requests
!git clone --depth 1 http://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2978, done.
remote: Counting objects: 100% (2978/2978), done.
remote: Compressing objects: 100% (2516/2516), done.
remote: Total 2978 (delta 509), reused 1890 (delta 386), pack-reused 0
Receiving objects: 100% (2978/2978), 376.92 MiB | 35.35 MiB/s, done.
Resolving deltas: 100% (509/509), done.
Checking out files: 100% (2809/2809), done.


In [0]:
models_path=os.path.join(os.getcwd(),'models')
sys.path.append(models_path)

from official.wide_deep import census_dataset
from official.wide_deep import census_main

census_dataset.download("/tmp/census_data/")

In [0]:
if "PYTHONPATH" in os.environ:
  os.environ['PYTHONPATH'] +=os.pathsep+models_path
else:
  os.environ['PYTHONPATH']=models_path
  

In [0]:
#!python -m official.wide_deep.census_main --help

In [9]:
!python -m official.wide_deep.census_main --model_type=wide --train_epochs=2

I1016 11:05:31.680608 139942071187328 tf_logging.py:115] Using config: {'_model_dir': '/tmp/census_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': device_count {
  key: "GPU"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f46a4a3abe0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
W1016 11:05:31.681873 139942071187328 tf_logging.py:120] 'cpuinfo' not imported. CPU info will not be logged.
I1016 11:05:36.726691 139942071187328 tf_logging.py:115] Benchmark run: {'model_name': 'wide_deep', '

In [10]:
!ls /tmp/census_data/

adult.data  adult.test


In [11]:
train_file="/tmp/census_data/adult.data"
test_file='/tmp/census_data/adult.test'

import pandas

train_df =pandas.read_csv(train_file,header=None,names=census_dataset._CSV_COLUMNS)
test_df=pandas.read_csv(test_file,header=None,names=census_dataset._CSV_COLUMNS)

train_df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [0]:
def easy_input_function(df,label_key,num_epochs,shuffle,batch_size):
  label=df[label_key]
  ds=tf.data.Dataset.from_tensor_slices((dict(df),label))
  
  if shuffle:
    ds=ds.shuffle(10000)
    
  ds=ds.batch(batch_size).repeat(num_epochs)
  
  return ds

ds=easy_input_function(train_df,)